In [43]:
import pandas as pd
import numpy as np
import os

In [47]:
df = pd.read_excel("/Users/jacopomichelacci/FINM_32500/data/proshares_analysis_data.xlsx", sheet_name='hedge_fund_series')
df = df.drop(columns=["Unnamed: 0"])

mean_r = df.mean() * 12
vol_r = df.std() * (12 ** 0.5)

rfr = 0.02

sharpe = (mean_r - rfr) / vol_r

print("Mean Annual Return: \n")
print(mean_r.round(4))

print("\nVolatility of Return: \n")
print(vol_r.round(4))

print("\nSharpe: \n")
print(sharpe.round(4))

Mean Annual Return: 

HFRIFWI Index     0.0513
MLEIFCTR Index    0.0385
MLEIFCTX Index    0.0365
HDG US Equity     0.0269
QAI US Equity     0.0288
dtype: float64

Volatility of Return: 

HFRIFWI Index     0.0588
MLEIFCTR Index    0.0552
MLEIFCTX Index    0.0551
HDG US Equity     0.0574
QAI US Equity     0.0498
dtype: float64

Sharpe: 

HFRIFWI Index     0.5320
MLEIFCTR Index    0.3355
MLEIFCTX Index    0.2999
HDG US Equity     0.1199
QAI US Equity     0.1769
dtype: float64


In [56]:
print("Part 2. \n\n")

df = pd.read_excel("/Users/jacopomichelacci/FINM_32500/data/proshares_analysis_data.xlsx", sheet_name='hedge_fund_series')
df.rename(columns={df.columns[0]: "datetime"}, inplace=True)
df.set_index("datetime", inplace=True)

def max_drawdown(ret_series):
    equity = (1 + ret_series).cumprod()
    peak = equity.cummax()
    dd = (equity - peak) / peak
    max_dd = min(dd)

    date_valley = dd.idxmin()
    date_peak = equity.loc[:date_valley].idxmax()
    value_peak1 = max(equity.loc[:date_valley])
    date_recovery = equity.loc[date_valley:].loc[equity >= value_peak1].index.min()

    return max_dd, date_valley, date_peak, date_recovery


skew = df.skew()
exc_kurt = df.kurt() - 3
VaR_05 = df.quantile(0.05)
CVaR_05 = df.apply(lambda x: x[x <= x.quantile(0.05)].mean())

#create lists to hold variables for each intrument
max_dd, date_valley, date_peak, date_recovery = [], [], [], []
for col in df.columns:
    max_dd1, date_valley1, date_peak1, date_recovery1 = max_drawdown(df[col])

    max_dd.append(max_dd1)
    date_valley.append(date_valley1)
    date_peak.append(date_peak1)
    date_recovery.append(date_recovery1)

#turn list into pd.series
max_dd = pd.Series(max_dd, index=df.columns)
date_valley = pd.Series(date_valley, index=df.columns)
date_peak = pd.Series(date_peak, index=df.columns)
date_recovery = pd.Series(date_recovery, index=df.columns)


df_result = pd.DataFrame({
    "skew" : skew,
    "exc_Kurt3" : exc_kurt,
    "VaR_.5" : VaR_05,
    "CVaR_.5" : CVaR_05,
    "max_dd" : max_dd,
    "date_valley" : date_valley,
    "date_peak" : date_peak,
    "date_recovery" : date_recovery
})

print(df_result)



Part 2. 


                    skew  exc_Kurt3    VaR_.5   CVaR_.5    max_dd date_valley  \
HFRIFWI Index  -0.948272   2.657351 -0.024008 -0.035992 -0.115473  2020-03-31   
MLEIFCTR Index -0.289982  -1.369083 -0.027002 -0.034993 -0.124302  2022-09-30   
MLEIFCTX Index -0.273477  -1.410210 -0.026970 -0.034937 -0.124388  2022-09-30   
HDG US Equity  -0.274904  -1.223519 -0.029919 -0.036831 -0.140720  2022-09-30   
QAI US Equity  -0.433508  -1.550816 -0.017171 -0.030995 -0.137716  2022-09-30   

                date_peak date_recovery  
HFRIFWI Index  2019-12-31    2020-08-31  
MLEIFCTR Index 2021-06-30    2024-02-29  
MLEIFCTX Index 2021-06-30    2024-02-29  
HDG US Equity  2021-06-30    2024-07-31  
QAI US Equity  2021-06-30    2024-02-29  


In [ ]:
print("Part 3. \n\n")